In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv('Heart_Disease.csv',header=None)
df.head()

Now we will give names to the coloumns

In [ ]:
df.columns=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','target']
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
sns.countplot(df['slope'])

In [ ]:
sns.countplot(df['ca'])

In [ ]:
sns.countplot(df['thal'])

In [ ]:
df.cloumns=[['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','target']]
df.head()

In [ ]:
df.shape

# Lets replace missing value with nan so that it will be easy for us to analyse

In [ ]:
df.replace({'?': np.nan}, regex=False,inplace=True)
df.isnull().sum()

first considering the nan values

In [ ]:
df.dropna(subset=["trestbps"], axis=0, inplace=True)
df.dropna(subset=["oldpeak"], axis=0, inplace=True)
df.dropna(subset=["chol"], axis=0, inplace=True)
df.dropna(subset=["fbs"], axis=0, inplace=True)
df.isnull().sum()

In [ ]:
df.dtypes

Conversion of value in target values distiguising the prensence value from the absence value

In [ ]:
df['target'] = df['target'].replace({2: 1})
df['target'] = df['target'].replace({3: 1})
df['target'] = df['target'].replace({4: 1})
df['target'].unique()

Now, our output will be clear, where 0 will indicate absence and 1 will indicate presence

# Converting whole data into a same datatype

In [ ]:
df=df.astype('float')
df.dtypes

Reseting the index 

In [ ]:
df.reset_index(drop=True, inplace=True)

checking for outliers

First we will check for outliers visually for each attributes, for this box plot is the best option

In [ ]:
col=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','target']
for i in range(0,11):
    print(sns.boxplot(df[col[i]]))
    plt.show()

We will romove the outliers as they are huge here

In [ ]:
from scipy.stats import zscore
import numpy as np
z = np.abs(zscore(df))
print('The shape of the dataset before removing outliers is',df.shape)
df_z = df[(z < 3).all(axis=1)]
print('The shape of the dataset after removing outliers is',df_z.shape)
df=df_z

In [ ]:
from scipy.stats import zscore
import numpy as np
z = np.abs(zscore(df))
print('The shape of the dataset before removing outliers is',df.shape)
df_z = df[(z < 3).all(axis=1)]
print('The shape of the dataset after removing outliers is',df_z.shape)
df=df_z

# Calculating correlation

In [ ]:
df.corr()

doing Visualisation with the help of heatmaps

In [ ]:
corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(12, 12))
    ax = sns.heatmap(corr, mask=mask, vmax=.3, square=True,annot=True, linewidths=0.5,cmap="YlGnBu")

finding out the coloumns which are not usefull for us as those columns are not corelated with output

In [ ]:
df1=df.corr()
#since we only care about the correlation with our output, lets separate it
df2=df1.iloc[:,10:11]
df2

If the value is 0 there is no correlation, if value is 1 they are highly correlated.

Lets set our threshold value as 0.1 and drop columns with value less than this considering they are not correlated

In [ ]:
np.abs(df2)<0.1

From the above data we can see that chol,restecg,thalach are not correlated with target, hence we can drop them

We are using regplot here to visualize the same. There is a line that passes through the points in the plot which will give us an idea if that column is positively correlated or negatively correlated or not correlated at all. If the line comes close to horizontal, we can assume that its not correlated and drop such columns as those columns are not helping us in any way to make our prediction.

In [ ]:
col=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','target']

for i in range(0,8):
    print(sns.regplot(x=col[i],y="target",data=df))
    plt.show()

# Separating our input and output features

In [ ]:
x=df[['age','sex','cp','trestbps','fbs','exang','oldpeak']]
x.head()

In [ ]:
y=df[['target']]
y.head()

# Lets check for skewness in our input features

In [ ]:
x.skew()

# Lets scale our data using standard scalar

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(x)
df_x=ss.transform(x)
df_x

In [ ]:
x=pd.DataFrame(df_x,columns=x.columns)
x.head()

We have both of input and output attributes cleaned and in desired format

End of EDA Process-
Lets start Building models to make predictions and find the model that works best on our dataset

Start of Machine Learning Process-
Since out target variable is Bivariant, we are going to do classification analysis

Lets import required packages

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc

# Lets split our data randomly and see which model works better

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

Lets import all the classification models

In [ ]:
KNN=KNeighborsClassifier(n_neighbors=6)
SV=SVC()
LR=LogisticRegression()
DT=DecisionTreeClassifier(random_state=6)
GNB=GaussianNB()

In [ ]:
models = []
models.append(('KNeighborsClassifier', KNN))
models.append(('SVC', SV))
models.append(('LogisticRegression', LR))
models.append(('DecisionTreeClassifier', DT))
models.append(('GaussianNB', GNB))

# Lets create a loop that will execute all our models

In [ ]:

Model = []
score = []
cvs=[]
rocscore=[]
for name,model in models:
    print('*-----------------------------*',name,'*------------------------------*')
    print('\n')
    Model.append(name)
    model.fit(x_train,y_train)
    print(model)
    pre=model.predict(x_test)
    print('\n')
    AS=accuracy_score(y_test,pre)
    print('Accuracy_score = ',AS)
    score.append(AS*100)
    print('\n')
    sc = cross_val_score(model, x, y, cv=10, scoring='accuracy').mean()
    print('Cross_Val_Score = ',sc)
    cvs.append(sc*100)
    print('\n')
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,pre)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    print ('roc_auc_score = ',roc_auc)
    rocscore.append(roc_auc*100)
    print('\n')
    print('classification_report\n',classification_report(y_test,pre))
    print('\n')
    cm=confusion_matrix(y_test,pre)
    print(cm)
    print('\n')
    plt.figure(figsize=(10,40))
    plt.subplot(911)
    plt.title(name)
    print(sns.heatmap(cm,annot=True))
    plt.subplot(912)
    plt.title(name)
    plt.plot(false_positive_rate, true_positive_rate, label='AUC = %0.2f'% roc_auc)
    plt.plot([0,1],[0,1],'r--')
    plt.legend(loc='lower right')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    print('\n\n')

In [ ]:
Final_R = pd.DataFrame({'Classification Model': Model, 'Accuracy Score': score ,'Cross_val_score':cvs,'Roc_auc_curve':rocscore})
Final_R

DecisonTree seems to have higher accuracy, lets try to make this better

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100)

We will take DecisionTreeClassifier as final model since accuracy is more

In [ ]:
Now lets save our final model

In [ ]:
import joblib
joblib.dump(DT,'Heart_Disease.pkl')